# Notebook 2 — Modeling Skeleton

This notebook shows steps for a small supervised POC: label generation, feature engineering, training a simple LightGBM classifier, and saving metrics & feature importance.

Run after `notebook_1` so `csv_files/merged_by_date.csv` and processed trades exist.

In [ ]:
# Example: Label next-day direction (pnl positive/negative)
import pandas as pd
merged = pd.read_csv('csv_files/merged_by_date.csv', parse_dates=['date'])
merged = merged.sort_values('date')
merged['pnl_next_1'] = merged['pnl_sum'].shift(-1)
merged['label'] = (merged['pnl_next_1'] > 0).astype(int)
merged.dropna(subset=['label'], inplace=True)
print('Labels distribution:\n', merged['label'].value_counts())

In [ ]:
# Feature engineering: add rolling features
merged['pnl_3d'] = merged['pnl_sum'].rolling(3, min_periods=1).mean()
merged['vol_3d'] = merged['total_volume'].rolling(3, min_periods=1).mean()
merged['sent_signal'] = merged['classification'].map({'greed':1,'fear':-1,'neutral':0}).fillna(0)

features = ['pnl_3d','vol_3d','avg_leverage','sent_signal']
X = merged[features].fillna(0)
y = merged['label']
print('X shape', X.shape)


In [ ]:
# Train simple LightGBM classifier (install in Colab if needed)
try:
    import lightgbm as lgb
except Exception:
    !pip install --quiet lightgbm
    import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
clf = lgb.LGBMClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, pred))
print('ROC AUC:', roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))

# Feature importance
import matplotlib.pyplot as plt
imp = pd.Series(clf.feature_importances_, index=features).sort_values(ascending=False)
imp.plot(kind='bar', figsize=(8,4))
plt.title('Feature importance')
plt.tight_layout()
plt.savefig('outputs/model_feature_importance.png')
plt.show()